# Importing packages

In [2]:
!pip install rioxarray stackstac pystac-client planetary-computer odc-stac rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 14.9 MB/s eta 0:00:00


In [3]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import RdYlGn,jet,RdBu

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

# Defining the boundaries which includes (Manhattan and Bronx)

In [4]:
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

In [5]:
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

## Loading the key bands from sentinel2 satellite with the help of microsoft planetary computer

In [6]:
import planetary_computer
import pystac_client
import stackstac

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(
    bbox=bounds,
    collections=["sentinel-2-l2a"],
    datetime="2021-06-01/2021-09-30",
    query={"eo:cloud_cover": {"lt": 10}},  # Filter for low cloud cover
)


In [7]:
# Get list of all matching items
items = list(search.get_all_items())

# Select the image with the least cloud cover
best_item = min(items, key=lambda x: x.properties["eo:cloud_cover"])

# Extract URLs for all available bands in the selected image
band_urls = [planetary_computer.sign(item).to_dict() for item in items]

print("All Band URLs:", band_urls)

All Band URLs: [{'type': 'Feature', 'stac_version': '1.1.0', 'stac_extensions': ['https://stac-extensions.github.io/eo/v1.1.0/schema.json', 'https://stac-extensions.github.io/sat/v1.0.0/schema.json', 'https://stac-extensions.github.io/projection/v2.0.0/schema.json'], 'id': 'S2A_MSIL2A_20210929T154041_R011_T18TWL_20210930T045940', 'geometry': {'type': 'Polygon', 'coordinates': [[[-74.88678, 40.5621133], [-74.86688, 40.6274513], [-74.82144, 40.7744995], [-74.77615, 40.9216841], [-74.730804, 41.0689543], [-74.68515, 41.2161276], [-74.63907, 41.3631148], [-74.59326, 41.5101582], [-74.58096, 41.5494454], [-73.68381, 41.5443115], [-73.70337, 40.5554731], [-74.88678, 40.5621133]]]}, 'bbox': [-74.88678, 40.5554731, -73.68381, 41.5494454], 'properties': {'datetime': '2021-09-29T15:40:41.024000Z', 'platform': 'Sentinel-2A', 'instruments': ['msi'], 's2:mgrs_tile': '18TWL', 'constellation': 'Sentinel 2', 's2:granule_id': 'S2A_OPER_MSI_L2A_TL_ESRI_20210930T045941_A032755_T18TWL_N03.00', 'eo:cloud_c

In [8]:
data = stac_load(
    items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    crs="EPSG:32632", # Latitude-Longitude
    resolution=10, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="float32",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

display(data)

print("Loaded Data:", data)

<xarray.Dataset> Size: 3GB
Dimensions:      (y: 2284, x: 2499, time: 11)
Coordinates:
  * y            (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x            (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06 -5.71e+06
    spatial_ref  int32 4B 32632
  * time         (time) datetime64[ns] 88B 2021-06-06T15:38:09.024000 ... 202...
Data variables:
    B01          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B02          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B05          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B06          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B07          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B8A          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B11          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B12          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

Loaded Data: <xarray.Dataset> Size: 3GB
Dimensions:      (y: 2284, x: 2499, time: 11)
Coordinates:
  * y            (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x            (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06 -5.71e+06
    spatial_ref  int32 4B 32632
  * time         (time) datetime64[ns] 88B 2021-06-06T15:38:09.024000 ... 202...
Data variables:
    B01          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B02          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B05          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B06          (time, y, x) float32 251MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B07          (ti

# Computing the median of bands across the time period

In [9]:
# Compute median composite across time dimension
data_median = data.median(dim="time", skipna=True)

print("Median Composite Computed")
display(data_median)

Median Composite Computed


<xarray.Dataset> Size: 251MB
Dimensions:      (y: 2284, x: 2499)
Coordinates:
  * y            (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x            (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06 -5.71e+06
    spatial_ref  int32 4B 32632
Data variables:
    B01          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B07          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B08          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B8A          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B11          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B12          (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>

# Feature enginnering

# NDVI (Normalized Difference Vegetation Index)


In [10]:
# Compute NDVI = (NIR - Red) / (NIR + Red)
ndvi = (data_median.B08-data_median.B04)/(data_median.B08+data_median.B04)

# Add NDVI to dataset
data_median["sentinel_NDVI"] = ndvi

print("NDVI Computed")
display(data_median)

NDVI Computed


<xarray.Dataset> Size: 274MB
Dimensions:        (y: 2284, x: 2499)
Coordinates:
  * y              (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x              (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref    int32 4B 32632
Data variables:
    B01            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B07            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B08            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B8A            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B11            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B12            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDVI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>

# NDBI (Normalized Difference Built-up Index)

In [11]:
ndbi_median = (data_median.B11-data_median.B08)/(data_median.B11+data_median.B08)

data_median["sentinel_NDBI"] = ndbi_median

print("NDBI Computed")

display(data_median)

NDBI Computed


<xarray.Dataset> Size: 297MB
Dimensions:        (y: 2284, x: 2499)
Coordinates:
  * y              (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x              (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref    int32 4B 32632
Data variables: (12/13)
    B01            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    ...             ...
    B08            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B8A            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B11            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B12            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDVI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDBI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>

# NDWI (Normalized Difference Water Index)

In [12]:
ndwi_median = (data_median.B03-data_median.B08)/(data_median.B03+data_median.B08)

data_median["sentinel_NDWI"] = ndwi_median

print("NDWI Computed")

display(data_median)

NDWI Computed


<xarray.Dataset> Size: 320MB
Dimensions:        (y: 2284, x: 2499)
Coordinates:
  * y              (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x              (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref    int32 4B 32632
Data variables: (12/14)
    B01            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    ...             ...
    B8A            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B11            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B12            (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDVI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDBI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDWI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>

# MNDWI (Modified Normalized Difference Water Index)

In [13]:
data_median['sentinel_MNDWI'] = (data_median['B03'] - data_median['B11']) / (data_median['B03'] + data_median['B11'])

print("sentinel_MNDWI")

display(data_median)

sentinel_MNDWI


<xarray.Dataset> Size: 343MB
Dimensions:         (y: 2284, x: 2499)
Coordinates:
  * y               (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x               (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref     int32 4B 32632
Data variables: (12/15)
    B01             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    ...              ...
    B11             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B12             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDVI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDBI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDWI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_MNDWI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>

# SAVI (Soil-Adjusted Vegetation Index)

In [14]:
data_median['sentinel_SAVI'] = ((data_median['B08'] - data_median['B04']) / (data_median['B08'] + data_median['B04'])) * 1.5

print("sentinel_SAVI")

display(data_median)

sentinel_SAVI


<xarray.Dataset> Size: 365MB
Dimensions:         (y: 2284, x: 2499)
Coordinates:
  * y               (y) float64 18kB 9.108e+06 9.108e+06 ... 9.086e+06 9.085e+06
  * x               (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref     int32 4B 32632
Data variables: (12/16)
    B01             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    ...              ...
    B12             (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDVI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDBI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_NDWI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_MNDWI  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_SAVI   (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>

# Albedo (Reflectivity of a Surface)

In [15]:
data_median['sentinel_Albedo'] = (0.356 * data_median['B02'] + 0.130 * data_median['B04'] + 0.373 * data_median['B08'] +
                0.085 * data_median['B11'] + 0.072 * data_median['B12'] - 0.018)


# Brightness, wetness and Greenness

In [16]:
import numpy as np
from sklearn.decomposition import PCA

# Select relevant bands for PCA directly from xarray Dataset
bands = ['B02', 'B03', 'B04', 'B08', 'B11', 'B12']
X = np.stack([data_median[band].values.flatten() for band in bands], axis=1)  # Convert to NumPy array

# Apply PCA to extract Brightness, Greenness, Wetness
pca = PCA(n_components=3)
pca_components = pca.fit_transform(X)

# Reshape back to original dimensions (assuming 2D spatial grid)
dims = data_median.dims  # Get original dimensions
pca_shape = data_median[bands[0]].shape  # Get shape of a single band

# Convert PCA outputs back to xarray format
data_median['sentinel_Brightness'] = (dims, pca_components[:, 0].reshape(pca_shape))
data_median['sentinel_Greenness'] = (dims, pca_components[:, 1].reshape(pca_shape))
data_median['sentinel_Wetness'] = (dims, pca_components[:, 2].reshape(pca_shape))

print(data_median)


<xarray.Dataset> Size: 457MB
Dimensions:              (y: 2284, x: 2499)
Coordinates:
  * y                    (y) float64 18kB 9.108e+06 9.108e+06 ... 9.085e+06
  * x                    (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref          int32 4B 32632
Data variables: (12/20)
    B01                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    ...                   ...
    sentinel_MNDWI       (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta

In [17]:
display(data_median)

<xarray.Dataset> Size: 457MB
Dimensions:              (y: 2284, x: 2499)
Coordinates:
  * y                    (y) float64 18kB 9.108e+06 9.108e+06 ... 9.085e+06
  * x                    (x) float64 20kB -5.735e+06 -5.735e+06 ... -5.71e+06
    spatial_ref          int32 4B 32632
Data variables: (12/20)
    B01                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B02                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B03                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B04                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B05                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    B06                  (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    ...                   ...
    sentinel_MNDWI       (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_SAVI        (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_Albedo      (y, x) float32 23MB dask.array<chunksize=(1669, 1826), meta=np.ndarray>
    sentinel_Brightness  (y, x) float32 23MB -408.5 -408.5 ... 1.154e+03 967.9
    sentinel_Greenness   (y, x) float32 23MB 506.8 506.8 133.4 ... -561.1 -516.2
    sentinel_Wetness     (y, x) float32 23MB 97.05 97.05 290.8 ... -287.0 -152.1

# Saving the data into Geotiff file

In [ ]:
import rioxarray as rio
import rasterio

# Define output filename
filename = "S2_median_composite.tiff"

# Ensure CRS is correctly assigned
data_median = data_median.rio.write_crs("EPSG:32632", inplace=True)

# Compute transformation matrix for georeferencing
height = data_median.dims["y"]
width = data_median.dims["x"]
gt = rasterio.transform.from_bounds(lower_left[1], lower_left[0], upper_right[1], upper_right[0], width, height)
data_median.rio.write_transform(transform=gt, inplace=True)

# Save the entire median composite with all computed bands
with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height,
                   crs='EPSG:32632', transform=gt, count=len(data_median.data_vars),
                   compress='lzw', dtype='float64') as dst:

    for i, band in enumerate(data_median.data_vars):  # Iterate through all available bands
        dst.write(data_median[band].values, i+1)

print(f"GeoTIFF saved successfully as: {filename}")

GeoTIFF saved successfully as: S2_median_composite.tiff


# uploading the Geotiff file into drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the Google Drive path where you want to save the file
drive_path = "/content/drive/My Drive/Colab Notebooks/S2_median_composite.tiff"

# Save the file to Google Drive
with rasterio.open(drive_path, 'w', driver='GTiff', width=width, height=height,
                   crs='EPSG:32632', transform=gt, count=len(data_median.data_vars),
                   compress='lzw', dtype='float64') as dst:

    for i, band in enumerate(data_median.data_vars):  # Iterate through all available bands
        dst.write(data_median[band].values, i+1)

print(f"GeoTIFF saved successfully at: {drive_path}")

GeoTIFF saved successfully at: /content/drive/My Drive/Colab Notebooks/S2_median_composite.tiff
